In [1]:
import requests
import csv

# Replace this with the URL of your "ventas" and "productos" API endpoints
ventas_url = "http://localhost:3001/ventas"
productos_url = "http://localhost:3001/productos"

response = requests.get(ventas_url)

if response.status_code == 200:
    ventas_data = response.json()
    if isinstance(ventas_data, list) and len(ventas_data) > 0:
        total_sum = 0
        best_selling_product = None

        for venta in ventas_data[0]:
            total_venta = float(venta.get("total_venta", 0))
            total_sum += total_venta

            if best_selling_product is None or total_venta > float(best_selling_product.get("total_venta", 0)):
                best_selling_product = venta

        print(f"Total sum of 'total_venta': {total_sum}")
        if best_selling_product:
            print(f"Best-selling product: {best_selling_product.get('nombre_producto')}")

            # Get the category of the best-selling product
            product_name = best_selling_product.get('nombre_producto')
            response = requests.get(productos_url)
            if response.status_code == 200:
                productos_data = response.json()
                category_name = None
                for productos_list in productos_data:
                    for producto in productos_list:
                        if producto.get("nombre_producto") == product_name:
                            category_name = producto.get("nombre_categoria")
                            print(f"Category of the best-selling product: {category_name}")
                            break
                    if category_name:
                        break

                if not category_name:
                    print("Category not found for the best-selling product.")
                else:
                    # Save data to CSV
                    csv_filename = "top_sold_product.csv"
                    with open(csv_filename, mode='w', newline='') as csv_file:
                        fieldnames = ["Total Sum of Sales", "Best-selling Product", "Category of the Best-Selling product"]
                        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

                        writer.writeheader()
                        writer.writerow({
                            "Total Sum of Sales": total_sum,
                            "Best-selling Product": best_selling_product.get('nombre_producto'),
                            "Category of the Best-Selling product": category_name
                        })

                    print(f"\nData saved to {csv_filename}")
            else:
                print("Failed to retrieve data from the 'productos' API")
        else:
            print("No best-selling product found in the 'ventas' data")
    else:
        print("No data found in the 'ventas' API response")
else:
    print("Failed to retrieve data from the 'ventas' API")


AttributeError: 'str' object has no attribute 'get'

In [5]:
import csv
import requests

# Define the URL for ventas and productos data
ventas_url = "http://localhost:3001/ventas/"
productos_url = "http://localhost:3001/productos/"

response_ventas = requests.get(ventas_url)
response_productos = requests.get(productos_url)

if response_ventas.status_code == 200 and response_productos.status_code == 200:
    ventas_data = response_ventas.json()
    productos_data = response_productos.json()[0]

    if isinstance(ventas_data, list) and len(ventas_data) > 0:
        total_sum = 0
        best_selling_product = None
        category_sales = {}

        for venta in ventas_data[0]:
            total_sale = float(venta.get("total_sale", 0))
            total_sum += total_sale

            # Collect category sales
            category = next((producto["nombre_categoria"] for producto in productos_data if producto["nombre_producto"] == venta["nombre_producto"]), None)
            if category:
                if category in category_sales:
                    category_sales[category] += total_sale
                else:
                    category_sales[category] = total_sale

            if best_selling_product is None or total_sale > float(best_selling_product.get("total_sale", 0)):
                best_selling_product = venta

        most_sold_category = max(category_sales, key=category_sales.get)

        print(f"Total sum of 'total_sale': {total_sum}")
        if best_selling_product:
            print(f"Best-selling product: {best_selling_product.get('nombre_producto')}")
            print(f"Most sold category: {most_sold_category}")

            # Saving data to CSV
            csv_filename = "top_sold_product.csv"
            with open(csv_filename, mode='w', newline='') as csv_file:
                fieldnames = ["Total Sum of Sales", "Best-selling Product", "Most Sold Category"]
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

                writer.writeheader()
                writer.writerow({
                    "Total Sum of Sales": total_sum,
                    "Best-selling Product": best_selling_product.get('nombre_producto'),
                    "Most Sold Category": most_sold_category
                })

            print(f"\nData saved to {csv_filename}")
        else:
            print("No best-selling product found in the data")
    else:
        print("No data found in the API response")
else:
    print("Failed to retrieve data from the API")


Total sum of 'total_sale': 3646.06
Best-selling product: Escape Suzuki Carbono Racing
Most sold category: Electronicos

Data saved to top_sold_product.csv
